In [1]:
Packages <- c("lmerTest", "arrow", "mgcv", "dplyr" , "effects")
invisible(lapply(Packages, library, character.only = TRUE))

Loading required package: lme4

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step



Attaching package: ‘arrow’


The following object is masked from ‘package:utils’:

    timestamp


Loading required package: nlme


Attaching package: ‘nlme’


The following object is masked from ‘package:lme4’:

    lmList


This is mgcv 1.9-0. For overview type 'help("mgcv-package")'.


Attaching package: ‘dplyr’


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: carData

lattice theme set by effectsTheme()
See ?effectsTheme for details.



In [2]:
packageVersion("rlang")

[1] ‘1.1.1’

In [3]:
datafolder <- '/home/hoyinchan/blue/Data/data2022/'

In [4]:
meds <- c('MEDATCA07DA',
 'MEDATCC03CA',
 'MEDATCC03CB',
 'MEDATCC03EB',
 'MEDATCC03AA',
 'MEDATCC03AB',
 'MEDATCC03AH',
 'MEDATCC03BA',
 'MEDATCC03EA',
 'MEDATCC09DX',
 'MEDATCC03DA',
 'MEDATCC03XA',
 'MEDATCC09XA',
 'MEDATCC03AX',
 'MEDATCC09BX',
 'MEDATCM05BA',
 'MEDATCM05BB')

In [5]:
#loincs <- c('28233', '178616', '29512', '21600')
# lab_loinc = {'2823-3':'potassium',
#             '17861-6':'calcium',
#             '2951-2':'sodium', 
#             '2160-0':'sCr'}


loincs <- c('LG49936-4', 'LG49864-8', 'LG11363-5', 'LG6657-3')
# lab_loinc = {'LG49936-4':'potassium',
#                 'LG49864-8':'calcium',
#                 'LG11363-5':'sodium', 
#                 'LG6657-3':'sCr'}

loincs <- gsub("-", "", loincs)

In [6]:
for (m in meds){
    for (l in loincs){
        tryCatch({
            path1 = paste0(datafolder,'xtitsa_',l,m,'.parquet')
            if (!file.exists(path1)) {
                cat(path1, "do not exists")
                next
            }
            
            df <- arrow::read_parquet(path1)
            df <- df %>% filter(DAYS_SINCE_ADMIT>=-7) %>% filter(DAYS_SINCE_ADMIT<=7)
            df$posttreatment <- as.factor(df$posttreatment)
#            x <- glm(RESULT_NUM ~  (1|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df)
            x <- lmer(RESULT_NUM ~  (1|ID) + (0 + DAYS_SINCE_ADMIT|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df, REML = FALSE,                       
                      control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B')))
            
            # x <- lmer(RESULT_NUM ~  (1|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df, REML = FALSE,                       
            #           control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B')))            
            print(paste(m, l, signif(fixef(x)[['posttreatment1']], digits = 2), coef(summary(x))['posttreatment1', 'Pr(>|t|)'], coef(summary(x))['posttreatment1', 'Pr(>|t|)']<0.05))
            print(paste(m, l, signif(fixef(x)[['posttreatment1:DAYS_SINCE_ADMIT']], digits = 2), coef(summary(x))['posttreatment1:DAYS_SINCE_ADMIT', 'Pr(>|t|)'], coef(summary(x))['posttreatment1:DAYS_SINCE_ADMIT', 'Pr(>|t|)']<0.05))
            flush.console()
            confx<-confint(x, method="Wald")
            df$predict <- predict(x, newdata = df, type = "response", re.form = NA)
            df$intercept <- fixef(x)[['posttreatment1']]
            df$interceptp <- coef(summary(x))['posttreatment1', 'Pr(>|t|)']
            df$slope <- fixef(x)[['posttreatment1:DAYS_SINCE_ADMIT']]
            df$slopep <- coef(summary(x))['posttreatment1:DAYS_SINCE_ADMIT', 'Pr(>|t|)']             
            df$interceptconfl <- confx['posttreatment1','2.5 %']
            df$interceptconfh <- confx['posttreatment1','97.5 %']
            df$slopeconfl <- confx['posttreatment1:DAYS_SINCE_ADMIT','2.5 %']
            df$slopeconfh <- confx['posttreatment1:DAYS_SINCE_ADMIT','97.5 %']            
            df$l <- l
            df$m <- m
#            path2 = paste0(datafolder,'xtitsa_result_',l,m,'.parquet')            
            path2 = paste0(datafolder,'xtitsa_result_norslope_',l,m,'.parquet')
            arrow::write_parquet(df, path2)
                },                 
#            print(paste(m, l, signif(fixef(x)[['posttreatment:DAYS_SINCE_ADMIT']], digits = 2)))},            
#        warning = function(w){},
        error = function(e) {print(paste(m, l, e))})
    }
}

Loading required namespace: optimx

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00605717 (tol = 0.002, component 1)”


[1] "MEDATCA07DA LG499364 0.017 9.22340403782062e-08 TRUE"
[1] "MEDATCA07DA LG499364 0.017 5.44461588213939e-65 TRUE"
[1] "MEDATCA07DA LG498648 0.001 0.759985797063547 FALSE"
[1] "MEDATCA07DA LG498648 0.042 5.25836444077053e-301 TRUE"
[1] "MEDATCA07DA LG113635 0.18 2.10281122045866e-20 TRUE"
[1] "MEDATCA07DA LG113635 0.039 2.55461231627529e-09 TRUE"
[1] "MEDATCA07DA LG66573 0.0031 0.00173718002174601 TRUE"
[1] "MEDATCA07DA LG66573 0.012 4.76694960984174e-286 TRUE"


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?”


[1] "MEDATCC03CA LG499364 -0.031 9.15423106447988e-79 TRUE"
[1] "MEDATCC03CA LG499364 0.0025 0.00023143123164548 TRUE"


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00212592 (tol = 0.002, component 1)”
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?”


...

[1] "MEDATCC03BA LG499364 -0.077 7.81973708526893e-40 TRUE"
[1] "MEDATCC03BA LG499364 0.029 1.33175298878123e-34 TRUE"
[1] "MEDATCC03BA LG498648 0.091 8.9257787993449e-75 TRUE"
[1] "MEDATCC03BA LG498648 0.054 6.09101337462531e-143 TRUE"
[1] "MEDATCC03BA LG113635 -0.45 1.71414970939021e-46 TRUE"
[1] "MEDATCC03BA LG113635 -0.16 1.95658757137194e-32 TRUE"
[1] "MEDATCC03BA LG66573 0.042 8.96882879705052e-72 TRUE"
[1] "MEDATCC03BA LG66573 0.015 5.79517774478504e-50 TRUE"
/home/hoyinchan/blue/Data/data2022/xtitsa_LG499364MEDATCC03EA.parquet do not exists/home/hoyinchan/blue/Data/data2022/xtitsa_LG498648MEDATCC03EA.parquet do not exists/home/hoyinchan/blue/Data/data2022/xtitsa_LG113635MEDATCC03EA.parquet do not exists/home/hoyinchan/blue/Data/data2022/xtitsa_LG66573MEDATCC03EA.parquet do not exists[1] "MEDATCC09DX LG499364 -0.0068 0.684440517214448 FALSE"
[1] "MEDATCC09DX LG499364 0.037 2.48218580502776e-06 TRUE"
[1] "MEDATCC09DX LG498648 -0.065 1.90834845205847e-05 TRUE"
[1] "MEDATCC09DX LG4

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00306662 (tol = 0.002, component 1)”


[1] "MEDATCC03DA LG113635 -0.24 2.73244080352777e-36 TRUE"
[1] "MEDATCC03DA LG113635 -0.29 1.27269265935827e-201 TRUE"
[1] "MEDATCC03DA LG66573 0.0094 3.25208936305268e-12 TRUE"
[1] "MEDATCC03DA LG66573 0.022 2.62728238831315e-240 TRUE"
[1] "MEDATCC03XA LG499364 -0.025 0.287579169824201 FALSE"
[1] "MEDATCC03XA LG499364 -0.049 2.77309663483304e-08 TRUE"
[1] "MEDATCC03XA LG498648 0.25 2.05611559988481e-33 TRUE"
[1] "MEDATCC03XA LG498648 0.091 8.0134350938883e-30 TRUE"
[1] "MEDATCC03XA LG113635 2.1 3.25517470206708e-33 TRUE"
[1] "MEDATCC03XA LG113635 1.8 1.51412332432177e-144 TRUE"
[1] "MEDATCC03XA LG66573 0.056 1.36255606085376e-17 TRUE"
[1] "MEDATCC03XA LG66573 0.0063 0.0138485761714209 TRUE"
[1] "MEDATCC09XA LG499364 -0.12 0.036985441391702 TRUE"
[1] "MEDATCC09XA LG499364 -0.057 0.145785141584169 FALSE"
[1] "MEDATCC09XA LG498648 -0.28 2.13264303682701e-05 TRUE"
[1] "MEDATCC09XA LG498648 0.0074 0.872582683789654 FALSE"
[1] "MEDATCC09XA LG113635 -0.044 0.897109247994976 FALSE"
[1] "MEDAT

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00207384 (tol = 0.002, component 1)”


[1] "MEDATCM05BA LG499364 -0.015 0.089548335126709 FALSE"
[1] "MEDATCM05BA LG499364 -0.0058 0.100196741290388 FALSE"
[1] "MEDATCM05BA LG498648 -0.14 4.63494994673907e-25 TRUE"
[1] "MEDATCM05BA LG498648 -0.054 3.49032312860716e-20 TRUE"
[1] "MEDATCM05BA LG113635 -0.11 0.0553299352925547 FALSE"
[1] "MEDATCM05BA LG113635 -0.083 0.000539634600036417 TRUE"
[1] "MEDATCM05BA LG66573 -0.00018 0.938665514621562 FALSE"
[1] "MEDATCM05BA LG66573 0.0087 9.09424740978141e-17 TRUE"
/home/hoyinchan/blue/Data/data2022/xtitsa_LG499364MEDATCM05BB.parquet do not exists/home/hoyinchan/blue/Data/data2022/xtitsa_LG498648MEDATCM05BB.parquet do not exists/home/hoyinchan/blue/Data/data2022/xtitsa_LG113635MEDATCM05BB.parquet do not exists/home/hoyinchan/blue/Data/data2022/xtitsa_LG66573MEDATCM05BB.parquet do not exists

In [7]:
print('done')

[1] "done"


In [ ]:
# l <- '21600'
# m <- 'MEDATCA07DA'

# path1 = paste0(datafolder,'xtitsa_',l,m,'.parquet')
# df <- arrow::read_parquet(path1)
# df <- df %>% filter(DAYS_SINCE_ADMIT>=-7) %>% filter(DAYS_SINCE_ADMIT<=7)
# df$posttreatment <- as.factor(df$posttreatment)
# #            x <- glm(RESULT_NUM ~  (1|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df)
# x <- lmer(RESULT_NUM ~  (1|ID) + (0 + DAYS_SINCE_ADMIT|ID) + posttreatment*DAYS_SINCE_ADMIT, data=df, REML = FALSE,                       
#           control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B')))

In [ ]:
# x

In [ ]:
# coef(x)$ID

In [ ]:
# summary(x)

In [ ]:
# confint(x, method="Wald")